**Data Preprocessing**

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Load your dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Separate the features and the target variable
X_train = train_df.drop('SalePrice', axis=1)
y_train = train_df['SalePrice']
X_test = test_df

# List of numerical and categorical features
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# Preprocessing for numerical data: impute missing values with median and scale them
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data: impute missing values with the most frequent value and apply one-hot encoding
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create a preprocessing and modeling pipeline
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Preprocess the training data
X_train_processed = model_pipeline.fit_transform(X_train)

# Preprocess the test data
X_test_processed = model_pipeline.transform(X_test)

# Now, X_train_processed and X_test_processed are ready for model training and predictions

**Feature Selection**

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression

# Load your dataset
train_df = pd.read_csv('train.csv')

# Separate the features and the target variable
X_train = train_df.drop('SalePrice', axis=1)
y_train = train_df['SalePrice']

# Identify categorical columns (assuming they are of type 'object')
categorical_cols = [cname for cname in X_train.columns if 
                    X_train[cname].dtype == "object"]

# Apply OneHotEncoder to categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

# Apply the preprocessor to the training data
X_train_encoded = preprocessor.fit_transform(X_train)

# Now we can safely perform feature selection
selector = SelectKBest(score_func=f_regression, k='all')
X_new = selector.fit_transform(X_train_encoded, y_train)

# Get the indices of the selected features
selected_features_indices = selector.get_support(indices=True)

# Get the names of the selected features
# Note: This will be the list of all new feature columns generated by OneHotEncoder
selected_features_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)

print("Selected features:", selected_features_names[selected_features_indices])

Selected features: ['MSZoning_C (all)' 'MSZoning_FV' 'MSZoning_RH' 'MSZoning_RL'
 'MSZoning_RM' 'Street_Grvl' 'Street_Pave' 'Alley_Grvl' 'Alley_Pave'
 'Alley_nan' 'LotShape_IR1' 'LotShape_IR2' 'LotShape_IR3' 'LotShape_Reg'
 'LandContour_Bnk' 'LandContour_HLS' 'LandContour_Low' 'LandContour_Lvl'
 'Utilities_AllPub' 'Utilities_NoSeWa' 'LotConfig_Corner'
 'LotConfig_CulDSac' 'LotConfig_FR2' 'LotConfig_FR3' 'LotConfig_Inside'
 'LandSlope_Gtl' 'LandSlope_Mod' 'LandSlope_Sev' 'Neighborhood_Blmngtn'
 'Neighborhood_Blueste' 'Neighborhood_BrDale' 'Neighborhood_BrkSide'
 'Neighborhood_ClearCr' 'Neighborhood_CollgCr' 'Neighborhood_Crawfor'
 'Neighborhood_Edwards' 'Neighborhood_Gilbert' 'Neighborhood_IDOTRR'
 'Neighborhood_MeadowV' 'Neighborhood_Mitchel' 'Neighborhood_NAmes'
 'Neighborhood_NPkVill' 'Neighborhood_NWAmes' 'Neighborhood_NoRidge'
 'Neighborhood_NridgHt' 'Neighborhood_OldTown' 'Neighborhood_SWISU'
 'Neighborhood_Sawyer' 'Neighborhood_SawyerW' 'Neighborhood_Somerst'
 'Neighborhood_Stone

In [4]:
from sklearn.model_selection import train_test_split
print("Length of X_train_processed:", X_train_processed.shape[0])
print("Length of y_train:", len(y_train))

# Ensure they have the same length
if X_train_processed.shape[0] == len(y_train):
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_processed, y_train, test_size=0.2, random_state=42)
else:
    print("The arrays do not have the same number of samples.")

Length of X_train_processed: 1460
Length of y_train: 1460


In [5]:
# Assuming 'train.csv' is loaded into train_df
train_df = pd.read_csv('train.csv')

# Assuming the preprocessing steps are correctly applied to X_train_processed
# Now, align y_train with the processed features
y_train_aligned = train_df['SalePrice']

# Check if the lengths match now
if X_train_processed.shape[0] == len(y_train_aligned):
    # Proceed with train-test split
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_processed, y_train_aligned, test_size=0.2, random_state=42)
else:
    print("There is still a mismatch in the number of samples.")

In [13]:
from sklearn.model_selection import cross_val_score
import xgboost as xgb

# Define the model
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 200)

# Perform cross-validation
scores = cross_val_score(xg_reg, X_train_processed, y_train_aligned, cv=5, scoring='neg_mean_squared_error')

# Calculate the mean RMSE across the cross-validated segments
mean_rmse = np.mean(np.sqrt(-scores))
print("Mean RMSE: %f" % (mean_rmse))

Mean RMSE: 26595.627261


**Model Training**: 

In [14]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Assuming X_train_processed and y_train_aligned are your preprocessed feature matrix and target vector
# Split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_processed, y_train_aligned, test_size=0.2, random_state=42)

# Instantiate an XGBoost regressor object with default parameters
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

# Fit the regressor to the training set
xg_reg.fit(X_train, y_train)

# Predict the labels of the validation set
y_pred = xg_reg.predict(X_valid)

# Compute and print the RMSE
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print("Validation RMSE: %f" % (rmse))

Validation RMSE: 27595.964851


**Model Evaluation*

In [15]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Assuming y_valid are the actual target values and y_pred are the predicted values from your model
rmse = sqrt(mean_squared_error(y_valid, y_pred))
print("RMSE on validation set: %f" % rmse)

RMSE on validation set: 27595.964851


In [17]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
import numpy as np

# Define the model
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror')

# Perform cross-validation
scores = cross_val_score(xg_reg, X_train_processed, y_train_aligned, scoring='neg_mean_squared_error', cv=5)

# Calculate the RMSE for each fold
rmse_scores = np.sqrt(-scores)

# Output the results
print("Cross-validated RMSE scores:", rmse_scores)
print("Mean RMSE:", rmse_scores.mean())

Cross-validated RMSE scores: [26499.67835039 34388.28826374 30499.55181851 22308.41424717
 31321.51474757]
Mean RMSE: 29003.48948547501


 **Prediction**

In [6]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Load the training data
train_data = pd.read_csv('train.csv')

# Separate target from predictors
y = train_data.SalePrice
X = train_data.drop(['SalePrice'], axis=1)

# Select categorical columns with relatively low cardinality (convenient for one-hot encoding)
categorical_cols = [cname for cname in X.columns if
                    X[cname].nunique() < 10 and 
                    X[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X.columns if 
                  X[cname].dtype in ['int64', 'float64']]

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define the model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Split data into train and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Load the test data
test_data = pd.read_csv('test.csv')

# Preprocessing of test data, fit model
preds_test = my_pipeline.predict(test_data)

# Save the predictions to a CSV file
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)